# HW3 鐵達尼號生存分析

## 載入模組

In [31]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import pandas as pd
import numpy as np
import talib as ta
import pandas_datareader as data
import pandas_datareader.data as web
import datetime
import math

## 載入資料

In [32]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv('gender_submission.csv')

## 觀察資料內容

In [65]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 查看缺失值

In [34]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [66]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


## 觀察資料分布狀況

In [35]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [36]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


## 選擇特徵

In [37]:
selected_features  = ['Pclass', 'Age', 'Fare', 'Sex', 'Embarked']

In [38]:
X_train = train[selected_features]

In [39]:
y_train = train['Survived']

In [40]:
X_test = test[selected_features]

## 處理缺失值

In [41]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
Pclass      891 non-null int64
Age         714 non-null float64
Fare        891 non-null float64
Sex         891 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 34.9+ KB


In [42]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 5 columns):
Pclass      418 non-null int64
Age         332 non-null float64
Fare        417 non-null float64
Sex         418 non-null object
Embarked    418 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 16.4+ KB


In [43]:
X_train['Age'].fillna(X_train['Age'].mean(), inplace=True)
X_train['Embarked'].fillna('S', inplace=True)
X_test['Age'].fillna(X_test['Age'].mean(), inplace=True)
X_test['Fare'].fillna(X_test['Fare'].mean(), inplace=True)

C:\Users\chenminluo\Anaconda3\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [44]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
Pclass      891 non-null int64
Age         891 non-null float64
Fare        891 non-null float64
Sex         891 non-null object
Embarked    891 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 34.9+ KB


## 將字串類型的類別資料做轉換

In [45]:
X_train = pd.get_dummies(X_train)

In [46]:
X_train.head()

,Pclass,Age,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,7.2500,0,1,0,0,1
1,1,38.0,71.2833,1,0,1,0,0
2,3,26.0,7.9250,1,0,0,0,1
3,1,35.0,53.1000,1,0,0,0,1
4,3,35.0,8.0500,0,1,0,0,1


In [47]:
X_test = pd.get_dummies(X_test)

In [48]:
X_test.head()

,Pclass,Age,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,7.8292,0,1,0,1,0
1,3,47.0,7.0000,1,0,0,0,1
2,2,62.0,9.6875,0,1,0,1,0
3,3,27.0,8.6625,0,1,0,0,1
4,3,22.0,12.2875,1,0,0,0,1


## 建立keras模型

In [49]:
model = Sequential()

In [56]:
model = Sequential()  
model.add(Dense(units=40,input_dim = 8,kernel_initializer='uniform',activation='relu'))  
model.add(Dense(30,activation='relu'))  
model.add(Dense(1,activation='sigmoid'))

In [57]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [58]:
train_his = model.fit(X_train ,y_train ,validation_split=0.1, epochs=100, batch_size=32, verbose=2)

Train on 801 samples, validate on 90 samples
Epoch 1/100
 - 0s - loss: 0.6504 - acc: 0.6567 - val_loss: 0.5810 - val_acc: 0.6667
Epoch 2/100
 - 0s - loss: 0.5980 - acc: 0.6916 - val_loss: 0.5498 - val_acc: 0.7111
Epoch 3/100
 - 0s - loss: 0.5826 - acc: 0.6941 - val_loss: 0.5294 - val_acc: 0.7111
Epoch 4/100
 - 0s - loss: 0.5782 - acc: 0.6816 - val_loss: 0.5172 - val_acc: 0.7333
Epoch 5/100
 - 0s - loss: 0.5591 - acc: 0.7016 - val_loss: 0.4916 - val_acc: 0.7333
Epoch 6/100
 - 0s - loss: 0.5400 - acc: 0.7166 - val_loss: 0.4979 - val_acc: 0.7556
Epoch 7/100
 - 0s - loss: 0.5533 - acc: 0.7129 - val_loss: 0.4738 - val_acc: 0.7444
Epoch 8/100
 - 0s - loss: 0.5193 - acc: 0.7303 - val_loss: 0.4680 - val_acc: 0.7667
Epoch 9/100
 - 0s - loss: 0.5034 - acc: 0.7566 - val_loss: 0.4654 - val_acc: 0.7667
Epoch 10/100
 - 0s - loss: 0.4905 - acc: 0.7715 - val_loss: 0.4554 - val_acc: 0.7667
Epoch 11/100
 - 0s - loss: 0.4845 - acc: 0.7790 - val_loss: 0.4638 - val_acc: 0.7667
Epoch 12/100
 - 0s - loss: 0.

Epoch 97/100
 - 0s - loss: 0.4348 - acc: 0.8052 - val_loss: 0.4126 - val_acc: 0.8333
Epoch 98/100
 - 0s - loss: 0.4314 - acc: 0.8077 - val_loss: 0.4055 - val_acc: 0.8333
Epoch 99/100
 - 0s - loss: 0.4282 - acc: 0.8140 - val_loss: 0.4051 - val_acc: 0.8000
Epoch 100/100
 - 0s - loss: 0.4233 - acc: 0.8127 - val_loss: 0.4137 - val_acc: 0.8111


In [59]:
scores = model.evaluate(X_train, y_train)

891/891 [==============================] - 0s 12us/step


In [60]:
survived_predict = model.predict(X_test)
survived_predict = np.array(survived_predict)
survived_predict = np.round(survived_predict)

In [62]:
survived_predict

array([[0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],

## 將結果輸出成規定的格式

In [63]:
submission=pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':survived_predict[0:418,0]})

In [64]:
submission.to_csv('submission.csv', index=False)